In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import zipfile
from datetime import datetime
import openassetpricing as openap

# -------------------- Parameters --------------------
signallist = ['IndIPO','BM']

# -------------------- Initialize OpenAP and Download Signal Doc --------------------
openap_obj = openap.OpenAP()  
signaldoc = openap_obj.dl_signal_doc('pandas')

print("Available signals:")
print(signaldoc[['Acronym', 'Authors', 'LongDescription']].head(20))

# -------------------- Download Portfolio Data --------------------
# Download OSAP portfolio returns for the IndIPO signal
port_osap = openap_obj.dl_port('op', 'pandas', signallist)
port_osap['date'] = pd.to_datetime(port_osap['date'])

# Reduce to LS portfolios (openap provides multiple portfolio versions for each anomaly on each date)
port_osap.query('port== "LS"', inplace=True)

# -------------------- Add Mkt-Rf returns --------------------

import pandas_datareader 
pandas_datareader.famafrench.get_available_datasets()
# Developed_3_Factors

# download F-F_Research_Data_Factors
ff = pandas_datareader.get_data_famafrench('F-F_Research_Data_Factors',start=1950,end=2024)[0]
ff = ff.reset_index().rename(columns={'Date':'yearm'})

# convert ff['Date'] from period[M] to YYYYMM number, to merge with port_osap
ff['yearm'] = ff['yearm'].dt.year * 100 + ff['yearm'].dt.month

# Create a mapping from dates in port_osap to a 'yearm' format (e.g. 202103 for March 2021)
date_mapping = port_osap[['date']].drop_duplicates().copy()
date_mapping['yearm'] = date_mapping['date'].dt.year * 100 + date_mapping['date'].dt.month
date_mapping, ff

# Merge into the FF data the port_osap dates for each money (last trading date of that month/year)
ff = pd.merge(ff, date_mapping, on='yearm', how='inner')

# For our purposes, we only need the date and the market excess return (Mkt-RF)
ff_processed = ff[['date', 'Mkt-RF']].copy()
ff_processed.rename(columns={'Mkt-RF': 'ret'}, inplace=True)
ff_processed['signalname'] = 'mkt_rf'
ff_processed

# -------------------- Combine Data -------------------- #
# NOTE: this drops all variables except date, ret, and signalname 

port_osap = pd.concat([port_osap[['date', 'ret', 'signalname']],
              ff_processed[['date', 'ret', 'signalname']]], ignore_index=True)

display(port_osap.groupby('signalname').describe()['ret'])

Available signals:
               Acronym                         Authors  \
0     AbnormalAccruals                             Xie   
1             Accruals                           Sloan   
2           AccrualsBM                  Bartov and Kim   
3            Activism1                Cremers and Nair   
4                   AM                 Fama and French   
5      AnalystRevision     Hawkins, Chamberlin, Daniel   
6   AnnouncementReturn  Chan, Jegadeesh and Lakonishok   
7          AssetGrowth        Cooper, Gulen and Schill   
8      BetaLiquidityPS            Pastor and Stambaugh   
9         BetaTailRisk                 Kelly and Jiang   
10             betaVIX                      Ang et al.   
11                  BM                        Stattman   
12               BMdec                 Fama and French   
13        BookLeverage                 Fama and French   
14               BPEBM     Penman, Richardson and Tuna   
15                Cash                         Palazz

/var/folders/nx/v58k3cwd0wlgff1pwy874p600000gn/T/ipykernel_51052/4133835328.py:36: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pandas_datareader.get_data_famafrench('F-F_Research_Data_Factors',start=1950,end=2024)[0]
/var/folders/nx/v58k3cwd0wlgff1pwy874p600000gn/T/ipykernel_51052/4133835328.py:36: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pandas_datareader.get_data_famafrench('F-F_Research_Data_Factors',start=1950,end=2024)[0]


,count,mean,min,25%,50%,75%,max,std
signalname,,,,,,,,
BM,870.0,0.674195,-16.609171,-1.605303,0.480837,2.524949,33.273039,4.122496
IndIPO,584.0,0.442920,-18.936894,-1.079318,0.351437,2.080434,15.404382,3.382486
mkt_rf,870.0,0.642598,-23.240000,-1.940000,1.025000,3.422500,16.100000,4.357549


In [ ]:
ff_processed